# [SU.8.1 ADAPTECCA SCENARIOS: use of regional climate change scenarios for impact and adaptation studies](https://web.unican.es/cursosdeveranoyextension/cursos-de-verano/curso?c=2861)

## Tools for accessing and processing climate data: Case study with R

### Herramientas para el acceso y procesamiento de datos climáticos: Caso práctico con R




****
![c4R](shared/figs/climate4R_esquema.png)

------------

This worked example contains the code that reproduces part of the examples shown in the paper ["climate4R: An R-based Framework for Climate Data Access, Post-processing and Bias Correction"](https://www.sciencedirect.com/science/article/pii/S1364815218303049).

In [ ]:
library(loadeR)
library(transformeR)
library(visualizeR)
library(downscaleR)
library(climate4R.climdex)

***
# 1. Cliamte data loading from OPeNDAP server: E-OBS observational data 
## Loading, collocating and harmonizing data 


The domain of the study area is defined by the following bounding coordinates:

In [ ]:
lon <- c(-10, 5)
lat <- c(36, 44)


The SU index (summer days) can be obtained on-the-fly by loading maximum temperature data with function `loadGridData` and by the following argument settings: `aggr.m = "sum"`, `condition = "GT"` and `threshold = 25`. First we load E-OBS observational data by pointing to a NetCDF file via OPeNDAP. Previous to loading, function `dataInventory` might be applied for an overview of the dataset, which returns an inventory (object `di`) of the available variables names, units, coordinates, etc.  


In [ ]:
eobs<-"http://opendap.knmi.nl/knmi/thredds/dodsC/e-obs_0.25regular/tx_0.25deg_reg_v17.0.nc"
di <- dataInventory(eobs)

In [ ]:
str(di)

In this case, the NetCDF file contains maximum temperature data named as "tx", thus, we set `var = "tx"` when calling to `loadGridData`:

In [ ]:
SU <- loadGridData(eobs, var = "tx",
                    season = 1:12, 
                    years = 1971:2000,
                    lonLim = lon, 
                    latLim = lat,
                    aggr.m = "sum", 
                    condition = "GT", 
                    threshold = 25)

In [ ]:
?loadGridData

#### Using a dictionary

In order to load and work with harmonized data we can repeat the above operation using a dictionary file, that defines the necessary name and unit transformations to the standard parameters. Function `C4R.vocabulary` displays the climate4R standard variable naming and units:


In [ ]:
C4R.vocabulary()

In this case, the only non-standard parameter in the E-OBS dataset is the variable name ("tx"), however, we could perform further loading requests using the standard name if a dictionary file is crated previously (see the [`loadeR` wiki](https://github.com/SantanderMetGroup/loadeR/wiki/Harmonization)). This can be done easily, for instance, in the following manner:

In [ ]:
file.create("dicEOBS.dic")
writeLines(c("identifier,short_name,time_step,lower_time_bound,upper_time_bound, cell_method,offset,scale,deaccum,derived,interface",
             "tasmax,tx,24h,0,24,max,0,1,0,0,"), "dicEOBS.dic")

Next the loading operation is repeated but using the standard name for the maximum temperature (`var = "tasmax"`) and by passing the path to our *.dic file ("dicEOBS.dic") in argument `dictionary`:

In [ ]:
SU <- loadGridData(eobs,
                         var = "tasmax",
                         season = 1:12,
                         lonLim = lon,
                         latLim = lat,
                         years = 1971:2000,
                         aggr.m = "sum", 
                         threshold = 25,
                         condition = "GT",
                         dictionary = "dicEOBS.dic")

#### Transformation and visualization

An useful plotting function is `temporalPlot` that displays temporal series of multiple datasets
and periods on the same plot. Here we plot the series corresponding to a single grid box (`latLim = 37.89`, `lonLim = -4.78`). If several grid boxes are considered (e.g. the whole domain) `temporalPlot` performs the spatial (`lat` and `lon` dimensions) aggregation before plotting (the `mean` is computed by default, type `?temporalPlot`).

Note that function `temporalPlot` is based on `lattice` and arguments from function `xyplot` are optionally passed to argument `xyplot.custom`, allowing for a fine tuning of multiple graphical parameters. 

In [ ]:
# Cordoba
temporalPlot(SU, lonLim = -4.78, latLim = 37.89, xyplot.custom = list(ylim = c(0, 40)))

In [ ]:
# Macizo de Aralar (cerca de San Sebastian)
temporalPlot(SU, lonLim = -2, latLim = 43, xyplot.custom = list(ylim = c(0, 40)))


Note that `loadGridData` returns monthly summer days (SU). To compute the annual index we only need to apply function `aggregateGrid` that performs the aggregation of the desired data dimension (in this case `time`). We use argument `aggr.y` to perform annual aggregation with function `sum`:


In [ ]:
SU.annual <- aggregateGrid(SU, aggr.y = list(FUN = "sum"))

In [ ]:
# Macizo de Aralar (cerca de San Sebastian)
temporalPlot(SU.annual, lonLim = -2, latLim = 43, xyplot.custom = list(ylim = c(15, 75)))

## ETCCDI index calculation (SU) from daily data

Daily data of the original variable is loaded as above but excluding arguments `aggr.m`, `threshold` y `condition`. 

In [ ]:
TX.obs <- loadGridData(eobs,
                         var = "tasmax",
                         season = 1:12,
                         lonLim = lon,
                         latLim = lat,
                         years = 1971:2000,
                         #aggr.m = "sum", 
                         #threshold = 25,
                         #condition = "GT",
                         dictionary = "dicEOBS.dic")

For plotting maps function `spatialPlot` is used. For plotting the maximum temperature here we select the `"Spectral"` color theme.

In [ ]:
spatialPlot(climatology(TX.obs), backdrop.theme = "countries",
            at = seq(5, 35, 1), color.theme = "Spectral")

Display the ETCCDI indices: 

In [ ]:
climdexShow()

Next the raw SU index (summer days) is calculated (object `SU.obs`), in a single line with function `climdexGrid`:

In [ ]:
SU.obs <- climdexGrid(tx = TX.obs, index.code = "SU")

In [ ]:
# Macizo de Aralar (cerca de San Sebastian)
temporalPlot(SU.obs, lonLim = -2, latLim = 43, xyplot.custom = list(ylim = c(15, 75)))

In this case we set the `"RdYlBu"` color theme to visualize the mean annual SU for the reference period (1971-2000).


In [ ]:
spatialPlot(climatology(SU.annual), backdrop.theme = "countries", 
            color.theme = "RdYlBu", rev.colors = TRUE,
            at = seq(0, 260, 10))

***
# 2. Cliamte data loading from local files: CORDEX historical projections
Next, projection data (for both the historical and the RCP8.5 scenarios) is loaded from local NetCDF files, which correspond to a particular RCM (Regional Climate Model ICHEC-EC-EARTH_r12i1p1_SMHI-RCA4_v1) from EURO-CORDEX. These files were downloaded from ESGF (see Appendix A in the manuscript) and stored locally. Next we list them in objects `dir` and `dirf`, the first corresponding to the historical scenario and the second to the future RCP8.5.

In [ ]:
dir <- "shared/local_nc_data/historical/"
dirf <- "shared/local_nc_data/rcp85/"
print(list.files(dir, recursive = T))

Each file in the list contains data for a 5-year period of the same variable (tasmax). Therefore, we use a "catalog" (*.ncml file) to load data for the required period without worrying about the different files that need to be read and bound. Next we create two catalogs (for each scenario) with function `makeAggregateDataset` ("CDX_hist.ncml" and "CDX_rcp85.ncml"):

In [ ]:
makeAggregatedDataset(source.dir = dir, recursive = T, ncml.file = "data/CDX_hist.ncml")
makeAggregatedDataset(source.dir = dirf, recursive = T, ncml.file = "data/CDX_rcp85.ncml")

The created *.ncml files are then used as a single access point to load data and to do the data inventory as well:

In [ ]:
di <- dataInventory("data/CDX_hist.ncml")
str(di$tasmax)

Contrarily to the case of the E-OBS dataset, the variable name is standard, but not the units (K). Therefore we define the harmonization parameters in another dictionary file ("dicCDX.dic"), where the offset is -273.15 to convert the data to the standard units (ºC):


In [ ]:
file.create("dicCDX.dic")
writeLines(c("identifier,short_name,time_step,lower_time_bound,upper_time_bound,cell_metod,offset,scale,deaccum,derived,interface",
             "tasmax,tasmax,24h,0,24,max,-273.15,1,0,0,"), "dicCDX.dic")

#### Historical data
Next, harmonized data is loaded for a single CORDEX model, for the historical scenario and the same reference period used to load E-OBS observational data (1971-2000):

In [ ]:
TX.hist <- loadGridData(dataset = "data/CDX_hist.ncml",
                     var = "tasmax",
                     season = 1:12,
                     lonLim = lon,
                     latLim = lat,
                     years = 1971:2000,
                     dictionary = "dicCDX.dic")

In [ ]:
spatialPlot(climatology(TX.hist), backdrop.theme = "countries",
            color.theme = "Spectral", rev.colors = TRUE,
            at = seq(5, 35, 1))

As can be noted in the resulting figure, the spatial grid of CORDEX is different from E-OBS. We can use function `interpGrid` to interpolate CORDEX data to the E-OBS spatial grid.

In [ ]:
TX.hist.i <- interpGrid(TX.hist, getGrid(TX.obs))


In [ ]:
spatialPlot(climatology(TX.hist.i), backdrop.theme = "countries",
            color.theme = "Spectral", rev.colors = TRUE,
            at = seq(5, 35, 1))

Nest we calculate the  `SU` index for CORDEX historical data

In [ ]:
SU.hist <- climdexGrid(tx = TX.hist.i, index.code = "SU")

In [ ]:
spatialPlot(climatology(SU.hist), backdrop.theme = "countries",             
            color.theme = "RdYlBu", rev.colors = TRUE,
           at = seq(0, 260, 10))

Despite not being necessary, here we apply a land mask before calculating the bias in order to eliminate the values projected by the CORDEX model over the sea. To do so, `gridArithmetics` can be used, first to create the mask and second to apply it.

In [ ]:
eobs.mask <- gridArithmetics(SU.obs, 0, operator = "*")
spatialPlot(climatology(eobs.mask))

In [ ]:
SU.hist <- gridArithmetics(SU.hist, eobs.mask, operator = "+")

In [ ]:
spatialPlot(climatology(SU.hist), backdrop.theme = "countries",             
            color.theme = "RdYlBu", rev.colors = TRUE,
           at = seq(0, 260, 10))

The bias is computed by subtracting the SU index of E-OBS (object `SU.obs`) to the SU index of historical CORDEX (object `SU.hist`), for which function `gridArithmetics` is again used. 

In [ ]:
bias <- gridArithmetics(SU.hist, SU.obs, operator = "-")

Next we plot the bias (object `bias`), in this case we select the `"PiYG"` color theme:

In [ ]:
spatialPlot(climatology(bias), backdrop.theme = "countries", 
            at = seq(-100, 100, 10), color.theme = "PiYG")

In [ ]:
# Macizo de Aralar (cerca de San Sebastian)
temporalPlot(SU.obs, SU.hist, xyplot.custom = list(ylim = c(50, 120)))

Lets also visualize the quantile quantile plot for a single location (Cordoba in this example). Function `subsetGrid` performs spatial subsetting by setting arguments `lonLim` and `latLim`. We can use function `qqplot` for plotting the quantiles of the historical CORDEX data vs observed data.

In [ ]:
obs1 <- subsetGrid(TX.obs, lonLim = -4.78, latLim = 37.89)
hist1 <- subsetGrid(TX.hist.i, lonLim = -4.78, latLim = 37.89)
qqplot(obs1$Data, hist1$Data)
lines(c(0, 50), c(0, 50), col = "red")

***
# 3. Cliamte data loading from UDG: CORDEX future climate change projections

Future projection can be loaded following the same steps followed above to load historical data. However, in order to illustrate the third option for data loading, here we load data for the RCP8.5 scenario and future period 2071-2100 via the Santander User Data Gateway (UDG).

Display available datasets:

In [ ]:
UDG.datasets(pattern = "EUR44.*rcp85.*RCA4")$name

login:

In [ ]:
loginUDG(username = "miturbide", password = "lukinvela9&9")

Load harmonized historical CORDEX data:

In [ ]:
TX.fut <- loadGridData(dataset = "CORDEX-EUR44_EC-EARTH_r12i1p1_rcp85_RCA4_v1",
                     var = "tasmax",
                     season = 1:12,
                     lonLim = lon,
                     latLim = lat,
                     years = 2071:2100,
                     #dictionary = "local_nc_data/dicCDX.dic"
                      )

We perfome the same operations of interpolation, index calculation and land/sea masking as before:

In [ ]:
TX.fut.i <- interpGrid(TX.fut, getGrid(TX.obs))

In [ ]:
SU.fut <- climdexGrid(tx = TX.fut.i, index.code = "SU")
SU.fut <- gridArithmetics(SU.fut, eobs.mask, operator = "+")

Note that in this case the application of `gridArithmetics` gives the projected climate change signal (object `CCsignal`) w.r.t the historical period (object `SUh.interp`).

In [ ]:

CCsignal <- gridArithmetics(SU.fut, 
                            SU.hist,
                            operator = "-")

Next we plot the SU index and the change signal:  

In [ ]:
spatialPlot(climatology(SU.fut), backdrop.theme = "countries", 
            color.theme = "RdYlBu", rev.colors = TRUE,
            at = seq(0, 260, 10))

In [ ]:
spatialPlot(climatology(CCsignal), backdrop.theme = "countries",
            at = seq(0, 80, 5), color.theme = "Reds")

***
# 4. Post-processing: Bias Correction


In [ ]:
# Media espacial
temporalPlot(SU.obs, SU.hist, SU.fut, 
             cols = c("black", "red", "red"),
             xyplot.custom = list(ylim = c(50, 170)))


Next the "eqm" method is applied to bias correct historical CORDEX data (object `TX.hist`) by means of function `biasCorrection`. 

In [ ]:
TX.hist.bc <- biasCorrection(y = TX.obs, x = TX.hist, 
                         method = "eqm")

qqplots:

In [ ]:
hist.bc1 <- subsetGrid(TX.hist.bc, lonLim = -4.78, latLim = 37.89)
par(mfrow = c(1, 2))
qqplot(obs1$Data, hist1$Data)
lines(c(0, 50), c(0, 50), col = "red")
qqplot(obs1$Data, hist.bc1$Data)
lines(c(0, 50), c(0, 50), col = "red")


Next the "eqm" method is applied to bias correct future CORDEX data (object `TX.fut`).

In [ ]:

TX.fut.bc <- biasCorrection(y = TX.obs, x = TX.hist, newdata = TX.fut, 
                         method = "eqm", extrapolation = "constant")

qqplots:

In [ ]:
fut1 <- subsetGrid(TX.fut.i, lonLim = -4.78, latLim = 37.89)
fut.bc1 <- subsetGrid(TX.fut.bc, lonLim = -4.78, latLim = 37.89)
par(mfrow = c(1, 2))
qqplot(obs1$Data, fut1$Data)
lines(c(0, 50), c(0, 50), col = "red")
qqplot(obs1$Data, fut.bc1$Data)
lines(c(0, 50), c(0, 50), col = "red")

Index calculation is repeated but for bias adjusted data:

In [ ]:
SU.hist.bc <- climdexGrid(tx = TX.hist.bc, index.code = "SU")
SU.fut.bc <- climdexGrid(tx = TX.fut.bc, index.code = "SU")

In [ ]:
# Media espacial
temporalPlot(SU.obs, SU.hist, SU.fut, SU.hist.bc, SU.fut.bc, 
             cols = c("black", "red", "red", "blue", "blue"),
             xyplot.custom = list(ylim = c(50, 170)))

Climate change signal calculation is also repeated for the corrected data:

In [ ]:
CCsignal.bc <- gridArithmetics(SU.fut.bc, 
                            SU.obs,
                            operator = "-")

By plotting the resulting objects we obtain...

In [ ]:
spatialPlot(climatology(SU.fut.bc), backdrop.theme = "countries", 
            at = seq(0, 260, 10), color.theme = "RdYlBu", rev.colors = TRUE)

In [ ]:
spatialPlot(climatology(CCsignal.bc), backdrop.theme = "countries",
            at = seq(0, 80, 5), color.theme = "Reds")

Export figures using functions `pdf` and `dev.off`:

In [ ]:
pdf("CCsignal.bc.pdf")
spatialPlot(climatology(CCsignal.bc), backdrop.theme = "countries",
            at = seq(0, 80, 5), color.theme = "Reds")
dev.off()

****
![results](shared/figs/final_figures.pdf.png)

# Other available material

* [2018_climate4R_example1.pdf](https://github.com/SantanderMetGroup/notebooks/blob/devel/2018_climate4R_example1.pdf) and [2018_climate4R_example2.pdf](https://github.com/SantanderMetGroup/notebooks/blob/devel/2018_climate4R_example2.pdf) contain the full code for **Examples 1 and 2** of the paper `climate4R: An Ecosystem of R packages for Climate Data Access, Post-processing and Bias Correction'.
* Find more worked examples on the utilization of climate4R packages in their respective GitHub **wiki**-s at [https://github.com/SantanderMetGroup](https://github.com/SantanderMetGroup):
    + [loadeR: https://github.com/SantanderMetGroup/loadeR/wiki](https://github.com/SantanderMetGroup/loadeR/wiki)
    + [transformeR: https://github.com/SantanderMetGroup/transformeR/wiki](https://github.com/SantanderMetGroup/transformeR/wiki)
    + [downscaleR: https://github.com/SantanderMetGroup/downscaleR/wiki](https://github.com/SantanderMetGroup/downscaleR/wiki)
    + [visualizeR: https://github.com/SantanderMetGroup/visualizeR/wiki](https://github.com/SantanderMetGroup/visualizeR/wiki) 


# Links for software installation
* Jupyter notebook: [https://test-jupyter.readthedocs.io/en/latest/install.html](https://test-jupyter.readthedocs.io/en/latest/install.html)
* R: [https://cran.r-project.org/doc/manuals/r-release/R-admin.html] (https://cran.r-project.org/doc/manuals/r-release/R-admin.html](https://cran.r-project.org/doc/manuals/r-release/R-admin.html)
* Rstudio: [https://www.rstudio.com/products/rstudio/download/](https://www.rstudio.com/products/rstudio/download/)

# climate4R installation 
[https://github.com/SantanderMetGroup/climate4R](https://github.com/SantanderMetGroup/climate4R)
